In [37]:
import pandas as pd
pd.set_option('max_colwidth', 100)

siads_csv = pd.read_csv('2024-10-17-SIADS_RelatorioInventario.csv', on_bad_lines='skip', encoding='utf-8')


uasg = ''
uasg_nome = ''
ug = ''
ug_nome = ''
ourg = ''

# Remover valores NaN da coluna 'Unnamed: 0' antes de aplicar .str.contains()
ug_info = siads_csv[siads_csv['MATERIAL:'].notna()]
ug_info = ug_info['26418 - INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO']
ug_info = ug_info[ug_info.notna()]
ug_info = ug_info.drop_duplicates()
ug_info = ug_info.rename("infos")
# ug_info = ug_info[ug_info['26418 - INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO'].str.contains('-')]
# ug_info.to_clipboard(index=False)
uasg = ug_info.iloc[0].split('-')
uasg_nome = uasg[1].strip()
uasg = uasg[0].strip()

ug = ug_info.iloc[1].split('-')
ug_nome = ug[1].strip()
ug = ug[0].strip()
# print(uasg, uasg_nome, ug, ug_nome)
# uasg = ug_info['26418 - INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO']

# Aplicar o filtro somente nas linhas que contém '-'
siads_csv = siads_csv[['Unnamed: 0', 'Unnamed: 12', 'Unnamed: 14']]
siads_csv = siads_csv[siads_csv['Unnamed: 0'].notna()]
siads_csv = siads_csv[siads_csv['Unnamed: 0'].str.contains('-')]
siads_csv = siads_csv.drop_duplicates()
# print(siads_csv['MATERIAL:'].str.contains('-'))
siads_csv.to_clipboard(index=False)
display(siads_csv)
print(len(siads_csv))

,Unnamed: 0,Unnamed: 12,Unnamed: 14
11,CONTA: 115610100 - MATERIAIS DE CONSUMO,NaN,NaN
12,SUBITEM: 14 - MATERIAL EDUCATIVO E ESPORTIVO,NaN,NaN
13,74691 - 000424218 - MEDALHA EM BRONZE COMEMORATIVA - 00001 - UN,19.0,"1,80"
18,"74691 - 000503231 - CRONOMETRO DIGITAL RESISTENTE AGUA/REFERENCIA 4TM11, 365510 A , - 00001",5.0,"71,85"
23,"74691 - 000819808 - APITO EM PLáSTICO, PROFISSIONAL, COM ARGOLA PARA FIXAçãO - 00001 - UN",11.0,"12,87"
...,...,...,...
2892,"74691 - 001727206 - ESTOJO CONFECCIONADO EM LONA IMPERMEÁVEL NA COR VERDE BA NDEIRA,",500.0,"7,00"
2897,"74691 - 001744844 - CANETA ECOLOGICA PERSONALIZADA COM ESCRITA CANETA ""CLICK"" (PONTA",890.0,"3,27"
2902,"74691 - 001978160 - AGENDA PLANNER, FORMATO FECHADO 16CM X 22CM CAPA CAPA DURA DE",385.0,"26,60"
2907,74691 - 001978225 - BLOCO DE ANOTAÇÕES INSTITUCIONAL EM FORMATO 10CM X 20CM CAPA E,378.0,"3,45"


377


In [42]:
siads_df = pd.DataFrame(columns=['uasg', 'uasg_nome', 'ug', 'ug_nome', 'conta', 'conta_nome', 'subitem', 'subitem_nome', 'uorg', 
                                 'codigo', 'descricao', 'qtd', 'valor', 'valor_total'])
new_row = {
    'uasg':uasg,
    'uasg_nome':uasg_nome,
    'ug':ug,
    'ug_nome':ug_nome,
    'conta':'',
    'conta_nome':'',
    'subitem':'',
    'subitem_nome':'',
    'uorg':'',
    'codigo':'',
    'descricao':'',
    'qtd':'',
    'valor':'',
    'valor_total':''
}

display(siads_df)

,uasg,uasg_nome,ug,ug_nome,conta,conta_nome,subitem,subitem_nome,uorg,codigo,descricao,qtd,valor,valor_total


In [44]:
for i in range(0, len(siads_csv)):
    print(siads_csv.iloc[0])
    siads_df.loc[len(siads_df)] = new_row

Unnamed: 0     CONTA: 115610100 - MATERIAIS DE CONSUMO
Unnamed: 12                                        NaN
Unnamed: 14                                        NaN
Name: 11, dtype: object
Unnamed: 0     CONTA: 115610100 - MATERIAIS DE CONSUMO
Unnamed: 12                                        NaN
Unnamed: 14                                        NaN
Name: 11, dtype: object
Unnamed: 0     CONTA: 115610100 - MATERIAIS DE CONSUMO
Unnamed: 12                                        NaN
Unnamed: 14                                        NaN
Name: 11, dtype: object
Unnamed: 0     CONTA: 115610100 - MATERIAIS DE CONSUMO
Unnamed: 12                                        NaN
Unnamed: 14                                        NaN
Name: 11, dtype: object
Unnamed: 0     CONTA: 115610100 - MATERIAIS DE CONSUMO
Unnamed: 12                                        NaN
Unnamed: 14                                        NaN
Name: 11, dtype: object
Unnamed: 0     CONTA: 115610100 - MATERIAIS DE CONSUMO


In [45]:
display(siads_df)

,uasg,uasg_nome,ug,ug_nome,conta,conta_nome,subitem,subitem_nome,uorg,codigo,descricao,qtd,valor,valor_total
0,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",,,,,,,,,,
1,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",,,,,,,,,,
2,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",,,,,,,,,,
3,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",,,,,,,,,,
4,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",,,,,,,,,,
373,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",,,,,,,,,,
374,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",,,,,,,,,,
375,151911,IFPE,26418,"INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO",,,,,,,,,,
